# High Performing Computing, Part III

## Numpy performance

In [1]:
import numpy as np

In [2]:
%%timeit
foo=np.arange(1,10001)

3.07 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [3]:
%%timeit
foo=list[(x for x in range(10000))]

445 ns ± 3.75 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [4]:
%%timeit
foo=[x for x in range(1,10001)]

150 µs ± 3.99 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


numpy arange is about 50 times faster than the list comprehension.

In [5]:
def naive_dist(p, q):
    square_distance = 0
    for p_i, q_i in zip(p, q):
        square_distance += (p_i - q_i) ** 2
    return square_distance ** 0.5
p = [i for i in range(1000)]
q = [i + 2 for i in range(1000)]

In [6]:
%%timeit
naive_dist(p, q)

42.3 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
def simple_numpy_dist(p, q):
    return (np.sum((p - q) ** 2)) ** 0.5
p = np.arange(1000); q = np.arange(1000) + 2

In [8]:
%%timeit
simple_numpy_dist(p, q)

5.66 µs ± 96.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [9]:
def numpy_norm_dist(p, q):
    return np.linalg.norm(p - q)

In [10]:
%%timeit
numpy_norm_dist(p,q)

3.51 µs ± 47.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


numpy function is about 12 times faster than python regular function

## Numpy’s dispatch mechanism

References:
- [Writing custom array containers](https://numpy.org/doc/stable/user/basics.dispatch.html#basics-dispatch)

A convenient pattern is to define a decorator `implements` that can be used to add functions to `HANDLED_FUNCTIONS`.

In [11]:
HANDLED_FUNCTIONS = {}
def implements(np_function):
    def decorator(func):
        HANDLED_FUNCTIONS[np_function] = func
        return func
    return decorator

In [12]:
from Contexts.Functor import ListFunctor

@implements(np.sum)
def sum(arr:ListFunctor):
    "Implementation of np.sum for list object"
    return arr

@implements(np.mean)
def mean(arr:ListFunctor):
    "Implementation of np.mean for list objects"
    return arr / len(arr)

@implements(np.fft.fft)
def fft(arr:ListFunctor):
    return arr

In [13]:
N=100000
a=[x for x in range(N)]
b=np.array(a,dtype=float)

In [14]:
f=ListFunctor([b])

In [15]:
%%timeit
np.sum(f)

47 µs ± 2.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [16]:
%%timeit
f.fmap(np.sum)

21.2 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [17]:
%%timeit
np.sum(b)

20.2 µs ± 564 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
%%timeit
np.sum(np.fromiter((x for x in range(N)),dtype=float))

4.55 ms ± 50.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit
np.fft.fft(b)

1.52 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
%%timeit
f.fmap(np.fft.fft)

1.39 ms ± 63.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Conclusions

- Small overhead on fmap, its performace is close to native numpy function call.
- iterator with numpy function is very slow

## Cupy
References:
- [Cupy github](https://github.com/cupy/cupy)
- [User Guide](https://docs.cupy.dev/en/stable/user_guide/index.html)

In [21]:
import numpy as np

In [22]:
%%timeit
x=np.arange(6).reshape(2,3).astype('f')
x.sum(axis=1)

2.42 µs ± 85.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [23]:
import cupy as cp

In [24]:
%%timeit
with cp.cuda.Device(0):
   x_on_gpu0 = cp.array([1, 2, 3, 4, 5])


59.3 µs ± 27.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
x_on_gpu=cp.array([1,2,3,4,5])

In [26]:
x_on_gpu0 = cp.array([1, 2, 3, 4, 5])
x_on_gpu0.device

<CUDA Device 0>

In [27]:
x_on_gpu.device

<CUDA Device 0>

In [28]:
%%timeit
x_on_gpu=cp.array([1,2,3,4,5])
cp.linalg.norm(x_on_gpu)

133 µs ± 31.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit
x_cpu = np.array([1,2,3,4,5])
l2_cpu = np.linalg.norm(x_cpu)

2.77 µs ± 45.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [30]:
%%timeit
x = cp.arange(600).reshape(20, 30).astype('f')
x.sum(axis=1)

64.7 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [31]:
%%timeit
x = cp.arange(600).reshape(20, 30).astype('f')
x.sum(axis=1)

61.1 µs ± 2.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Conclusions
- not sure how cupy can help

## Dask
References:
- [Dask github](https://github.com/dask/dask)
- [Dask](https://examples.dask.org/array.html)

In [32]:
npx = np.random.random((10000,10000))

In [33]:
%%timeit
y=npx+npx.T
z=y[::2,5000:].mean(axis=1)

258 ms ± 3.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Dashboard

In [34]:
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://198.18.195.28:8787/status,
Dashboard: http://198.18.195.28:8787/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: False
Comm: inproc://198.18.195.28/54063/1,Workers: 1
Dashboard: http://198.18.195.28:8787/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: inproc://198.18.195.28/54063/4,Total threads: 4
Dashboard: http://198.18.195.28:43171/status,Memory: 1.86 GiB
Nanny: None,


In [35]:
import dask.array as da
x = da.random.random((10000, 10000), chunks=(1000, 1000))
x

2023-02-14 10:05:58,214 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 1.50 GiB -- Worker memory limit: 1.86 GiB
2023-02-14 10:05:58,215 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.50 GiB -- Worker memory limit: 1.86 GiB


dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

In [36]:
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z

dask.array<mean_agg-aggregate, shape=(5000,), dtype=float64, chunksize=(500,), chunktype=numpy.ndarray>

In [37]:
%%timeit
y = x + x.T
z = y[::2, 5000:].mean(axis=1)

1.58 ms ± 41.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [38]:
y.persist()

2023-02-14 10:06:11,444 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.61 GiB -- Worker memory limit: 1.86 GiB


dask.array<add, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

In [39]:
%%timeit
y[0, 0].compute()

2023-02-14 10:06:21,453 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.61 GiB -- Worker memory limit: 1.86 GiB
2023-02-14 10:06:31,554 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.61 GiB -- Worker memory limit: 1.86 GiB
2023-02-14 10:06:41,653 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

In [ ]:
%%timeit
y.sum().compute()

2023-02-14 10:02:51,242 - distributed.worker.memory - WARNING - Worker is at 75% memory usage. Resuming worker. Process memory: 1.41 GiB -- Worker memory limit: 1.86 GiB


The slowest run took 29.99 times longer than the fastest. This could mean that an intermediate result is being cached.
99.2 ms ± 193 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


2023-02-14 10:02:59,742 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.43 GiB -- Worker memory limit: 1.86 GiB
2023-02-14 10:03:09,843 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.43 GiB -- Worker memory limit: 1.86 GiB
2023-02-14 10:03:19,844 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

## Numba

## MKL

## Open-BLAS